In [3]:
# !pip install sentence-transformers
# !pip install faiss-gpu
# !pip install faiss-cpu
# !pip install tqdm
# !pip install nvidia-ml-py3
# !pip install accelerate
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 304.9 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e1770f6943a8515a2542d66dd278b102588935861ac99539172fbd6a8b02b95c
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 12.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19190 sha256=4f5454fa13ce6935d18b7362448033656fe4c532b777f64a0203b3c2eec7a05a
  Stored in directory: /root/.cache/pip/wheels/74/d2/c1/2ea351258984f451bd34e5ff2928621ab030e2eda5ffd2fdec
Successfully built nvidia-ml-py3


In [ ]:
!pip install --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/sentence-transformers-2.2.2.tar.gz --no-index
!pip install faiss-gpu --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/faiss_cpu-1.7.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-index
!pip install faiss-cpu --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/faiss_gpu-1.7.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-index
!pip install tqdm --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/tqdm-4.64.1-py2.py3-none-any.whl --no-index
!pip install nvidia-ml-py3 --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/nvidia-ml-py3-7.352.0.tar.gz --no-index
!pip install accelerate --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/accelerate-0.16.0-py3-none-any.whl --no-index
!pip install scikit-learn --find-links /kaggle/input/using-fine-tuned-sentencetransformer-env/scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-index

In [4]:
import os
import gc
import time
import math
import random
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, models, InputExample, losses, util
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoConfig, AutoTokenizer
import torch 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
!ls -l

total 4
---------- 1 root root 263 Mar  1 15:44 __notebook_source__.ipynb


In [17]:
Refresh_Topic = True
Refresh_Train_Data = True
Train_model = True
Build_Embedding = True 
Calculate_Score = True 
Cal_Submission = True
Cosine_Cutoff = 0.99995
dver = 505

Topic_Full_Data_File = f"/kaggle/working/df_topics_full_{dver}.csv"
Train_Data_File= f"/kaggle/working/df_train_v{dver}.csv"
Embeddings_File = f'/kaggle/working/embeddings_topics_dataset_v{dver}'

Submission_File = f"/kaggle/working/submission_v{dver}.csv"
Model_File = f"/kaggle/working/paraphrase-multilingual-mpnet-base-v{dver}-exp"


In [18]:
# pd.set_option('display.max_columns', 9)
# pd.set_option('display.max_rows', 200)
# pd.set_option('display.min_rows', 10)
# pd.set_option("expand_frame_repr", True)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', 50)
DATA_PATH = "/kaggle/input/learning-equality-curriculum-recommendations/"
# DATA_PATH = "./data/input/learning-equality-curriculum-recommendations/"
topics = pd.read_csv(DATA_PATH + "topics.csv")
content = pd.read_csv(DATA_PATH + "content.csv")
correlations = pd.read_csv(DATA_PATH + "correlations.csv")
# sample_submission = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [19]:
df_topics = None

if ( not topics.columns[0].startswith("topic_")):
    print(f"renaming topics ...")
    topics.rename(columns=lambda x: "topic_" + x, inplace=True)
    content.rename(columns=lambda x: "content_" + x, inplace=True)

renaming topics ...


In [ ]:
def get_topic_full(row):
    topic_title = str(row["topic_title"]) if pd.notna(row["topic_title"]) else ""
    topic_description = str(row["topic_description"]) if pd.notna(row["topic_description"]) else ""
    topic_full = "title: " + topic_title
    if (topic_description != ""):
        topic_full = topic_full + "\r\n" + "description: " + topic_description

    return topic_full

def get_parents(df, row):
    topic_id = row["topic_id"]
    topic_title_full = str(row["topic_title"]) if pd.notna(row["topic_title"]) else ""
    topic_parent = row["topic_parent"]
    topic_level = row["topic_level"]
    topic_full = get_topic_full(row)
    while not pd.isnull(topic_parent):
        subset = df.loc[df['topic_id'] == topic_parent]
        for index, r in subset.iterrows():
            t_full = get_topic_full(r)
            topic_full = t_full + "\r\n" + topic_full 
            t_title = str(row["topic_title"]) if pd.notna(row["topic_title"]) else ""
            topic_title_full = t_title + "." + topic_title_full 
            topic_parent = r["topic_parent"]
            topic_level = r["topic_level"]
            break

    return topic_title_full, topic_full


def refresh_topic(topics):

    df_topics = topics
    print(f"Before expand topic full ...")
    print(df_topics.head())

    topic_title_full = []
    topic_full = []


    for index, row in tqdm(df_topics.iterrows(), total=df_topics.shape[0]):
        t_title_full, t_full = get_parents(df_topics, row)
        topic_title_full.append(t_title_full)
        topic_full.append(t_full)

    df_topics['topic_title_full'] = topic_title_full
    df_topics['topic_full'] = topic_full

    df_topics.to_csv(Topic_Full_Data_File) 
    return df_topics

if (Refresh_Topic):
    print("Freshing topic...")
    df_topics = refresh_topic(topics)
else:
    df_topics = pd.read_csv(Topic_Full_Data_File) 

In [ ]:
print(f"After expand topic full ...")
print(df_topics.head())
print(f"df_topics value counet for each columns: \n{df_topics.nunique()}")

Load train data by combining correlation table with topic and content tables.

In [ ]:
def load_train_data(topics):
    train_df_columns = ["topic_title", "content_title", "topic_title_full", "topic_full", "topic_id","content_id", "content_description", "content_text" ]
    
    correlations["content_id"] = correlations["content_ids"].str.split(" ")
    corr = correlations.explode("content_id").drop(columns=["content_ids"])

    corr = corr.merge(df_topics, how="left", on="topic_id")
    corr = corr.merge(content, how="left", on="content_id")

    train_df = pd.DataFrame(corr[train_df_columns])
    cols = ['content_title', 'content_description', "content_text"]
    train_df['content_full'] = train_df[cols].apply(lambda row: '\r\n'.join(row.values.astype(str)), axis=1)
    
    final_train_data = pd.DataFrame(train_df)
    
    print(f"final_train_data value counet for each columns: \n{final_train_data.nunique()}")

    final_train_data.to_csv(Train_Data_File)
    
    return final_train_data

In [ ]:
if Refresh_Train_Data:
    print(f"Refresh_Train_Data ==>>>")
    final_train_data = load_train_data(topics)
else:
    print(f"load final_train_data from {Train_Data_File}")
    final_train_data = pd.read_csv(Train_Data_File)


In [ ]:
dataset_dict = load_dataset("csv", data_files=Train_Data_File)
print(dataset_dict)

dataset = dataset_dict["train"]

dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]

keeped_columns = ["topic_full", "content_full"]
columns = train_dataset.column_names
columns_to_keep = ["topic_full", "content_full"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
train_dataset = train_dataset.remove_columns(columns_to_remove)

# train_dataset = train_dataset.shuffle(seed=42).select(range(10000))

train_examples = []
eval_examples = []

n_examples = train_dataset.num_rows

In [ ]:
for dt in train_dataset:
    # print([dt["topic_full"], dt["content_full"]])
    # train_examples.append(InputExample(texts=[str(dt["topic_full"]), str(dt["content_full"])]))
    first = str(dt["topic_full"]) if dt["topic_full"] != "" else "" 
    second = str(dt["content_full"]) if str(dt["content_full"]) != "" else "" 
    if (first != "" and second != ""):
        train_examples.append(InputExample(texts=[first, second]))

print(f"train_examples: {len(train_examples)}")

In [ ]:
def train_model():

    model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    train_loss = losses.MultipleNegativesRankingLoss(model=model)
    # num_epochs = 5
    num_epochs = 1
    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

    model.to(device)
    gc.collect()
    torch.cuda.empty_cache()

    model.fit(train_objectives=[(train_dataloader, train_loss)],
            epochs=num_epochs,
            warmup_steps=warmup_steps)

    gc.collect()
    torch.cuda.empty_cache()

    model.save(Model_File)

In [ ]:
if Train_model:
    train_model()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Model_File)
trained_model = AutoModel.from_pretrained(Model_File)

trained_model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = trained_model(**encoded_input)
    return cls_pooling(model_output)


In [ ]:
# build embedding based on topic_full and content_full, we also need keep topic_id and content_id.
def build_embedding(combined_df):
    all_columns = combined_df.columns
    keeped_columns = ["topic_id","topic_full", "content_full","content_id"]
    removed_columns = all_columns.symmetric_difference(keeped_columns)
    combined_df.drop(removed_columns, axis=1, inplace = True)
    print(f"combined_df ==> {combined_df.head()}")  
    print(f"combined_df value counet for each columns: \n{combined_df.nunique()}")
    
    combined_df.dropna(inplace = True)
    # combined_df.dropna(subset = ["topic_title_full", "topic_full"], inplace = True)
    print(f"combined_df ==> {combined_df.shape}")  
    print(f"combined_df {combined_df.head()}")

    print(f"building embeddings for topic_full")
    embeddings_topics_dataset = Dataset.from_pandas(combined_df)
    embeddings_topics_dataset = embeddings_topics_dataset.map(
        lambda x: {"embeddings": get_embeddings( str(x["topic_full"])).detach().cpu().numpy()[0]}
    )

    print(f"building embeddings for content_full")
    conts = combined_df.copy()
    
    embeddings_content_dataset = Dataset.from_pandas(conts)
    embeddings_content_dataset = embeddings_content_dataset.map(
        lambda x: {"embeddings": get_embeddings( [str(x["content_full"])]).detach().cpu().numpy()[0]}
    )
    embeeding_dataset =  concatenate_datasets([embeddings_topics_dataset, embeddings_content_dataset])

    embeeding_dataset.save_to_disk(Embeddings_File)

In [ ]:
if Build_Embedding:
    print(f"Build Embedding .... {final_train_data.shape}")
    build_embedding(final_train_data)

print(f"final_train_data value counet for each columns: \n{final_train_data.nunique()}")
embeeding_dataset = load_from_disk(Embeddings_File)
print(embeeding_dataset)
embeeding_dataset.add_faiss_index(column="embeddings")

In [ ]:
def get_cosine_sim(ebd, text_embedding):
    cosine_score = util.pytorch_cos_sim(text_embedding, ebd)
    return cosine_score.item()

def get_score_topic(text, column_list, cosine_cutoff):
    text_embedding = get_embeddings(text).cpu().detach().numpy()
    scores, samples = embeeding_dataset.get_nearest_examples(
                                                    "embeddings", text_embedding, k=10
                                                    )
    # print(scores)
    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    samples_df.sort_values("scores", ascending=True, inplace=True)
    
    # embedding_list = samples_df["embeddings"].values.tolist()
    # cosine_sim_list = []
    # for ebd in embedding_list:
    #     cosine_score = util.pytorch_cos_sim(text_embedding, ebd)
    #     cosine_sim_list.append(cosine_score.item())

    samples_df['cosine_sim'] = samples_df.apply(lambda row: get_cosine_sim(row["embeddings"], text_embedding), axis=1)
    samples_df = samples_df[samples_df['cosine_sim'] >= cosine_cutoff] 

    scorelist = samples_df["scores"].values.tolist()
    # scores = " ".join([f"{d:.10f}" for d in scorelist])
    cosine_sim_list = samples_df["cosine_sim"].values.tolist()
    # cosine_sim = " ".join([f"{d:.10f}" for d in cosine_sim_list])
    if (column_list is not None):
        result_df = samples_df[['scores', 'cosine_sim', *column_list]]
    else:
        result_df = samples_df

    # content_id_list = samples_df["content_id"].values.tolist()
    # content_ids = " ".join(content_id_list)
    # topic_id_list = samples_df["topic_id"].values.tolist()
    # topic_ids = " ".join(topic_id_list)

    # return scores, topic_ids, content_ids, cosine_sim
    return result_df

def get_score_topic_json(text, column_list, cosine_cutoff):

    res_df = get_score_topic(text, column_list, cosine_cutoff)

    # scorelist = samples_df["scores"].values.tolist()
    # # scores = " ".join([f"{d:.10f}" for d in scorelist])
    # cosine_sim_list = samples_df["cosine_sim"].values.tolist()
    # # cosine_sim = " ".join([f"{d:.10f}" for d in cosine_sim_list])
    # result_df = samples_df

    # content_id_list = samples_df["content_id"].values.tolist()
    # content_ids = " ".join(content_id_list)
    # topic_id_list = samples_df["topic_id"].values.tolist()
    # topic_ids = " ".join(topic_id_list)

    res_dict = res_df.to_dict('records')
    return res_dict

In [ ]:
def calculate_row(row):
    columns = ["content_id", "topic_id"]
    res_dict = get_score_topic_json(str(row["topic_full"]), columns, Cosine_Cutoff) 
    # print(f"calculate_row dict")
    # print(res_dict)
    #scores, topic_ids, content_ids, cosine_sim
    # result.append([scores, topic_id, topic_title, **evalrow ])
    row["scores"] = " ".join([f"{d['scores']:.10f}" for d in res_dict])
    row["topic_ids"] = " ".join([f"{d['topic_id']}" for d in res_dict])
    row["content_ids"] = " ".join([f"{d['content_id']}" for d in res_dict])
    row["cosine_sim"] = " ".join([f"{d['cosine_sim']:.10f}" for d in res_dict])
    return row

def calculate_score(eval_sampled, res_file):
    num_rows = eval_sampled.num_rows
    scores_column = [-2] * num_rows
    topic_ids_column = [""] * num_rows
    content_ids_column = [""] * num_rows
    cosine_column = [-1.0] * num_rows
    eval_sampled = eval_sampled.add_column("scores", scores_column)
    eval_sampled = eval_sampled.add_column("topic_ids", topic_ids_column)
    eval_sampled = eval_sampled.add_column("content_ids", content_ids_column)
    eval_sampled = eval_sampled.add_column("cosine_sim", cosine_column)

    print(f"calculating scores mapping ...")
    eval_sampled = eval_sampled.map(calculate_row)
    # print(eval_sampled['scores'][:15])
    # print(eval_sampled['topic_ids'][:15])
    result_df = pd.DataFrame(eval_sampled)
    # print(result_df.head())
    print(f"result_df value counet for each columns: \n{result_df.nunique()}")
    all_columns = result_df.columns
    keeped_columns = ["topic_id","scores", "cosine_sim", "topic_ids","content_ids"]
    removed_columns = all_columns.symmetric_difference(keeped_columns)
    result_df.drop(removed_columns, axis=1, inplace=True)
    result_df.dropna( inplace=True)
    result_df.to_csv(res_file) 
    # df   ["topic_title", "content_title", "topic_title_full", "topic_description_full","topic_id","content_id"

Result_file = f"./data/result_v{dver}.csv"
if Calculate_Score:
    eval_sampled =  dataset["test"].shuffle(seed=42).select(range(2000))
    calculate_score(eval_sampled, Result_file)
    Result_file = f"./data/result_train_v{dver}.csv"
    eval_sampled =  dataset["train"].shuffle(seed=42).select(range(2000))
    calculate_score(eval_sampled, Result_file)

calc_df = pd.read_csv(Result_file) 


In [ ]:
def get_neighbors(topic_id, df_topics):
    subset = df_topics.loc[df_topics['topic_id'] == topic_id]
    text = None
    for index, r in subset.iterrows():
            text = r["topic_full"]
            break
    if text is None:
        return ""

    columns = ["content_id", "topic_id"]
    res_dict = get_score_topic_json(text, columns, Cosine_Cutoff) 

    content_ids = " ".join([f"{d['content_id']}" for d in res_dict])
    return content_ids


def calculate_submission(submission_df):

    submission_df['content_ids'] = submission_df.apply(lambda row: get_neighbors(row["topic_id"], df_topics), axis=1)
    submission_df.to_csv(DATA_PATH + "submission.csv") 
    submission_df.to_csv(Submission_File) 
    


In [ ]:
if (Cal_Submission):
    print(f"Calculating submission ")
    submission_df = pd.read_csv(DATA_PATH + "sample_submission.csv")
    calculate_submission(submission_df)

# from sklearn.metrics import fbeta_score
# fbeta_score(y_true, y_pred, average='macro', beta=2)